In [ ]:
import os
from data_loader.calib.intrinsic_extrinsic_loader import IntrinsicExtrinsicLoader
from data_loader.file_loader import FileLoader
from data_loader.file_writer import FileWriter
from cfg.dataset.cfg_sequence import dataset_sequence_calib_used_dict
from tools.utils import *
import cv2

platform = 'handheld'
sequence_name = 'handheld_room00'
algorithm = 'fastlio2'

##### Set up the output data path
dataset_path = '/Rocket_ssd/dataset/data_FusionPortable/sensor_data'
calib_data = dataset_sequence_calib_used_dict[sequence_name][1]
calib_path = os.path.join(dataset_path, '../calibration_files', calib_data, 'calib')
kitti360_path = os.path.join(dataset_path, sequence_name, 'kitti360')

In [ ]:
##### Set up the message topic list for different platforms
# Platform
if platform == 'handheld':
  from cfg.dataset.cfg_handheld import dataset_sensor_frameid_dict
  from cfg.dataset.cfg_handheld import dataset_rostopic_msg_frameid_dict
elif platform == 'ugv':
  from cfg.dataset.cfg_ugv import dataset_sensor_frameid_dict
  from cfg.dataset.cfg_ugv import dataset_rostopic_msg_frameid_dict
elif platform == 'legged':
  from cfg.dataset.cfg_legged import dataset_sensor_frameid_dict
  from cfg.dataset.cfg_legged import dataset_rostopic_msg_frameid_dict
  if sequence_name == 'legged_grass00':
    dataset_sensor_frameid_dict, dataset_rostopic_msg_frameid_dict = \
      filter_sensor('event', dataset_sensor_frameid_dict, dataset_rostopic_msg_frameid_dict)
elif platform =='vehicle':
  from cfg.dataset.cfg_vehicle import dataset_sensor_frameid_dict
  from cfg.dataset.cfg_vehicle import dataset_rostopic_msg_frameid_dict

# Algorithm
if algorithm == 'r3live':
  from cfg.algorithm.cfg_r3live import algorithm_rostopic_msg_frameid_dict
elif algorithm == 'fastlio2':
  from cfg.algorithm.cfg_fastlio2 import algorithm_rostopic_msg_frameid_dict

##### Set up the sensor configuration
int_ext_loader = IntrinsicExtrinsicLoader(is_print=False)
int_ext_loader.load_calibration(calib_path=calib_path, sensor_frameid_dict=dataset_sensor_frameid_dict)
print('Finish loading calibration parameters')

data_path = os.path.join(dataset_path, sequence_name, 'raw_data')
alg_result_path = os.path.join(dataset_path, sequence_name, 'algorithm_result')
file_loader = FileLoader()
file_writer = FileWriter()

##### Create the output data path
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'calibration'))
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'ouster00_undistort/points/data'))
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'ouster00/points/data'))
if platform == 'vehicle':
  flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'vehicle_frame_cam00/image/data'))
  flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'vehicle_frame_cam01/image/data'))
else:
  flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'frame_cam00/image/data'))
  flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'frame_cam01/image/data'))

flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'event_cam00/image/data'))
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'event_cam00/event/data'))
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'event_cam00/event_render/data'))
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'event_cam01/image/data'))
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'event_cam01/event/data'))
flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'event_cam01/event_render/data'))

flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'odometry_alg'))

if algorithm == 'r3live':
  flag = os.system('mkdir -p '+ os.path.join(kitti360_path, 'camera_odometry_alg'))
print('Finish creating Folder')

##### Visualize TF-tree
int_ext_loader.tf_graph.visualize_graph()

In [ ]:
##### Save calibration files according to KITTI format
calib_file_path = os.path.join(kitti360_path, 'calibration', 'perspective.txt')
file_writer.write_kitti_calibration_camera_intrinsics(platform, int_ext_loader, calib_file_path)
calib_file_path = os.path.join(kitti360_path, 'calibration', 'calib_cam_to_pose.txt')
file_writer.write_kitti_calibration_camera_extrinsics(platform, int_ext_loader, calib_file_path)

In [ ]:
##### Parameters
t_add = 0.1 # the timestamp of the ouster00_undistorted is at the time of the last point approximate to 0.08s

##### Load timestamp from alg_result
# Ouster
if 'ouster_points' in dataset_rostopic_msg_frameid_dict.keys():
  ouster_timestamps = file_loader.load_timestamp(os.path.join(data_path, 'ouster00/points/timestamps.txt')) 
  print('Loading ouster_timestamps: {}'.format(len(ouster_timestamps)))

##### Load timestamp from raw_data
# Ouster_undistort at the ouster00 frame
if 'ouster_points_undistorted' in algorithm_rostopic_msg_frameid_dict.keys():
  ouster_un_timestamps = file_loader.load_timestamp(os.path.join(alg_result_path, 'ouster00_undistort/points/timestamps.txt'))
  print('Loading ouster_un_timestamps: {}'.format(len(ouster_un_timestamps)))

# Frame_camera
if platform =='vehicle':
  if 'vehicle_frame_left_image' in dataset_rostopic_msg_frameid_dict.keys():
    frame_left_image_timestamps = file_loader.load_timestamp(os.path.join(data_path,'vehicle_frame_cam00/image/timestamps.txt'))
    print('Loading vehicle_left_timestamps: {}'.format(len(frame_left_image_timestamps)))
  if 'vehicle_frame_right_image' in dataset_rostopic_msg_frameid_dict.keys():
    frame_right_image_timestamps = file_loader.load_timestamp(os.path.join(data_path,'vehicle_frame_cam01/image/timestamps.txt'))
    print('Loading vehicle_right_timestamps: {}'.format(len(frame_right_image_timestamps)))
else:
  if 'frame_left_image' in dataset_rostopic_msg_frameid_dict.keys():
    frame_left_image_timestamps = file_loader.load_timestamp(os.path.join(data_path, 'frame_cam00/image/timestamps.txt'))
    print('Loading frame_left_timestamps: {}'.format(len(frame_left_image_timestamps)))
  if 'frame_right_image' in dataset_rostopic_msg_frameid_dict.keys():
    frame_right_image_timestamps = file_loader.load_timestamp(os.path.join(data_path, 'frame_cam01/image/timestamps.txt'))
    print('Loading frame_right_timestamps: {}'.format(len(frame_right_image_timestamps)))
    
# Event_camera
if 'event_left_image' in dataset_rostopic_msg_frameid_dict.keys():
  event_left_timestamps = file_loader.load_timestamp(os.path.join(data_path, 'event_cam00/event/timestamps.txt'))
  print('Loading event_left_timestamps: {}'.format(len(event_left_timestamps)))
  event_left_image_timestamps = file_loader.load_timestamp(os.path.join(data_path, 'event_cam00/image/timestamps.txt'))
  print('Loading event_left_image_timestamps: {}'.format(len(event_left_image_timestamps)))
else:
  event_left_timestamps = []
  event_left_image_timestamps = []

if 'event_right_image' in dataset_rostopic_msg_frameid_dict.keys():
  event_right_timestamps = file_loader.load_timestamp(os.path.join(data_path, 'event_cam01/event/timestamps.txt'))
  print('Loading event_right_timestamps: {}'.format(len(event_right_timestamps)))
  event_right_image_timestamps = file_loader.load_timestamp(os.path.join(data_path, 'event_cam01/image/timestamps.txt'))
  print('Loading event_right_image_timestamps: {}'.format(len(event_right_image_timestamps)))
else:
  event_right_timestamps = []
  event_right_image_timestamps = []

if 'odometry' in algorithm_rostopic_msg_frameid_dict.keys():
  odom_timestamps, odom_quats, odom_trans = file_loader.load_odometry(os.path.join(alg_result_path, 'odometry/odometry.txt'), traj_type='TUM')
  print('Loading odom_timestamps: {}'.format(len(odom_timestamps)))

if 'camera_odometry' in algorithm_rostopic_msg_frameid_dict.keys():
  camodom_timestamps, camodom_quats, camodom_trans = file_loader.load_odometry(os.path.join(alg_result_path, 'camera_odometry/odometry.txt'), traj_type='TUM')
  print('Loading camodom_timestamps: {}'.format(len(camodom_timestamps)))

##### Match synchronized timestamps
th_ouster_ousterun = 0.02
th_ouster_fc, th_fl_fr = 0.03, 0.03
th_ouster_ec, th_el_er = 0.02, 0.02
matched_id_ouster_sensors_odom = []

for id, time in enumerate(ouster_timestamps):
    ouster_un_time, ousterun_id = find_closest_element_sorted(ouster_un_timestamps, time + t_add)
    if ousterun_id is None or abs(time + t_add - ouster_un_time) >= th_ouster_ousterun:
        continue

    odom_time, odom_id = find_closest_element_sorted(odom_timestamps, time)
    if odom_id is None or abs(time - odom_time) >= th_ouster_ousterun:
        continue

    fl_time, fl_id = find_closest_element_sorted(frame_left_image_timestamps, time)
    fr_time, fr_id = find_closest_element_sorted(frame_right_image_timestamps, time)
    if fl_id is None or fr_id is None or abs(time - fl_time) >= th_ouster_fc or abs(time - fr_time) >= th_ouster_fc or abs(fl_time - fr_time) >= th_fl_fr:
        continue

    el_img_time, el_img_id = find_closest_element_sorted(event_left_image_timestamps, time)
    er_img_time, er_img_id = find_closest_element_sorted(event_right_image_timestamps, time)
    if el_img_id is not None and er_img_id is not None:
        if abs(time - el_img_time) < th_ouster_fc and abs(time - er_img_time) < th_ouster_fc:
            matched_id = [id, ousterun_id, odom_id, fl_id, fr_id, el_img_id, er_img_id]

    matched_id_ouster_sensors_odom.append(matched_id)

print('Length of matched id (ouster_un - sensors): {}'.format(len(matched_id_ouster_sensors_odom)))
print(f'{len(matched_id_ouster_sensors_odom)} x {len(matched_id_ouster_sensors_odom[0])}')

In [ ]:
##### Analyze matched timestamps
if len(matched_id_ouster_sensors_odom) > 0:
  import matplotlib.pyplot as plt
  num = [i for i in range(len(matched_id_ouster_sensors_odom))]
  ##### Ouster_undistorted
  dt_ouster_ousterun = [1e3 * abs(ouster_timestamps[mid[0]] + t_add - ouster_un_timestamps[mid[1]]) for mid in matched_id_ouster_sensors_odom]
  plt.plot(num, dt_ouster_ousterun, label='Ouster-OusterUn', marker='o', markersize=2, linestyle='none')
  ##### Odometry
  # dt_ouster_odo = [1e3 * abs(ouster_timestamps[mid[0]] - odom_timestamps[mid[2]]) for mid in matched_id_ouster_sensors_odom]
  # plt.plot(num, dt_ouster_odo, label='Ouster-odo', marker='o', markersize=2, linestyle='none') 
  ##### Frame camera
  dt_ouster_fl = [1e3 * abs(ouster_timestamps[mid[0]] - frame_left_image_timestamps[mid[3]]) for mid in matched_id_ouster_sensors_odom]
  plt.plot(num, dt_ouster_fl, label='Ouster-FL', marker='o', markersize=2, linestyle='none')
  dt_ouster_fr = [1e3 * abs(ouster_timestamps[mid[0]] - frame_right_image_timestamps[mid[4]]) for mid in matched_id_ouster_sensors_odom]
  plt.plot(num, dt_ouster_fr, label='Ouster-FR', marker='o', markersize=2, linestyle='none')
  dt_fl_fr = [1e3 * abs(frame_left_image_timestamps[mid[3]] - frame_right_image_timestamps[mid[4]]) for mid in matched_id_ouster_sensors_odom]
  plt.plot(num, dt_fl_fr, label='FL-FR', marker='o', markersize=2, linestyle='none')
  ##### Event camera
  if len(matched_id_ouster_sensors_odom[0]) >= 7:
    dt_ouster_el = [1e3 * abs(ouster_timestamps[mid[0]] - event_left_image_timestamps[mid[5]]) for mid in matched_id_ouster_sensors_odom]
    plt.plot(num, dt_ouster_el, label='Ouster-EL', marker='*', markersize=2, linestyle='none')
    dt_ouster_er = [1e3 * abs(ouster_timestamps[mid[0]] - event_right_image_timestamps[mid[6]]) for mid in matched_id_ouster_sensors_odom]
    plt.plot(num, dt_ouster_er, label='Ouster-Er', marker='*', markersize=2, linestyle='none')
    dt_el_er = [1e3 * abs(event_left_image_timestamps[mid[5]] - event_right_image_timestamps[mid[6]]) for mid in matched_id_ouster_sensors_odom]
    plt.plot(num, dt_el_er, label='EL-ER', marker='d', markersize=2, linestyle='none')

  plt.xlabel('Num') 
  plt.ylabel('Delta Timestamp [ms]')
  plt.legend(loc='upper right', bbox_to_anchor=(1.38, 1.02))
  plt.grid(True)  
  plt.show()

In [ ]:
##### Writing sensor data to files
def rewrite_undistort_image(old_path, new_path, camera):
  import cv2
  img = cv2.imread(old_path)
  undistorted_img = camera.undistort(img)
  cv2.imwrite(new_path, undistorted_img)
  return img, undistorted_img

In [ ]:
dt_ev_img = 0.03  # 30ms

def write_timestamps(timestamps, path):
	file_writer.write_timestamp(timestamps, path)

def write_odometry(timestamps, quats, trans, path, traj_type):
	file_writer.write_odometry(timestamps, quats, trans, path, traj_type)

def copy_file(src, dst):
	os.system(f'cp {src} {dst}')

def render_events(event_img, event_data, camera, ref_time, dt_ev_img):
	event_img_raw = np.ones_like(event_img) * 255
	selected_ids = [i for i, timestamp in enumerate(event_data['event_msg_timestamps']) if abs(ref_time - timestamp) <= dt_ev_img]

	for id in selected_ids:
		ev_data_path = os.path.join(event_data['path'], f'{id:06d}.txt')
		ev_data = np.loadtxt(ev_data_path)
		for ev in ev_data:
			if abs(ref_time - ev[0]) <= dt_ev_img / 2:
				color = [255, 0, 0] if ev[3] == 1 else [0, 0, 255] # BGR: blue for positive, red for negative
				event_img_raw[int(ev[2]), int(ev[1])] = color
				event_img[int(ev[2]), int(ev[1])] = color

	undistorted_event_img_raw = camera.undistort(event_img_raw)
	undistorted_event_img = camera.undistort(event_img)

	return undistorted_event_img_raw, undistorted_event_img

# Save timestamps
write_timestamps([ouster_timestamps[mid[0]] for mid in matched_id_ouster_sensors_odom], os.path.join(kitti360_path, 'ouster00/points/timestamps.txt'))
write_timestamps([ouster_un_timestamps[mid[1]] for mid in matched_id_ouster_sensors_odom], os.path.join(kitti360_path, 'ouster00_undistort/points/timestamps.txt'))

# Save odometry
odom_time = [odom_timestamps[mid[2]] for mid in matched_id_ouster_sensors_odom]
odom_q = [odom_quats[mid[2]] for mid in matched_id_ouster_sensors_odom]
odom_t = [odom_trans[mid[2]] for mid in matched_id_ouster_sensors_odom]
write_odometry(odom_time, odom_q, odom_t, os.path.join(kitti360_path, 'odometry_alg/odometry.txt'), traj_type='KITTI')

# Save frame camera timestamps
cam_prefix = 'vehicle_' if platform == 'vehicle' else ''
write_timestamps([frame_left_image_timestamps[mid[3]] for mid in matched_id_ouster_sensors_odom], os.path.join(kitti360_path, f'{cam_prefix}frame_cam00/image/timestamps.txt'))
write_timestamps([frame_right_image_timestamps[mid[4]] for mid in matched_id_ouster_sensors_odom], os.path.join(kitti360_path, f'{cam_prefix}frame_cam01/image/timestamps.txt'))

# Save event camera timestamps and render events
if event_left_image_timestamps and event_right_image_timestamps:
	for cam_idx, cam_name in enumerate(['event_cam00', 'event_cam01']):
		select_time = [event_left_image_timestamps[mid[5 + cam_idx]] for mid in matched_id_ouster_sensors_odom]
		write_timestamps(select_time, os.path.join(kitti360_path, f'{cam_name}/image/timestamps.txt'))
		write_timestamps(select_time, os.path.join(kitti360_path, f'{cam_name}/event/timestamps.txt'))
		write_timestamps(select_time, os.path.join(kitti360_path, f'{cam_name}/event_render/timestamps.txt'))

# Save sensor data
for frame_id, mid in enumerate(matched_id_ouster_sensors_odom):
	# Ouster data
	copy_file(os.path.join(data_path, 'ouster00/points/data', f'{mid[0]:06d}.pcd'), os.path.join(kitti360_path, 'ouster00/points/data', f'{frame_id:06d}.pcd'))
	copy_file(os.path.join(alg_result_path, 'ouster00_undistort/points/data', f'{mid[1]:06d}.pcd'), os.path.join(kitti360_path, 'ouster00_undistort/points/data', f'{frame_id:06d}.pcd'))

	# Frame camera data
	for cam_idx, cam_name in enumerate(['frame_cam00', 'frame_cam01']):
		cam_prefix = 'vehicle_' if platform == 'vehicle' else ''
		data_path_src = os.path.join(data_path, f'{cam_prefix}{cam_name}/image/data', f'{mid[3 + cam_idx]:06d}.png')
		data_path_dst = os.path.join(kitti360_path, f'{cam_prefix}{cam_name}/image/data', f'{frame_id:06d}.png')
		camera_type = 'frame_left_camera' if cam_name == 'frame_cam00' else 'frame_right_camera'
		camera = int_ext_loader.sensor_collection[f'{cam_prefix}{camera_type}']
		rewrite_undistort_image(data_path_src, data_path_dst, camera)

	# Event camera data
	if event_left_image_timestamps and event_right_image_timestamps:
		event_img_timestamps = [event_left_image_timestamps, event_right_image_timestamps]
		event_timestamps = [event_left_timestamps, event_right_timestamps]
		for cam_idx, cam_name in enumerate(['event_cam00', 'event_cam01']):
			# Event image
			data_path_src = os.path.join(data_path, f'{cam_name}/image/data', f'{mid[5 + cam_idx]:06d}.png')
			data_path_dst = os.path.join(kitti360_path, f'{cam_name}/image/data', f'{frame_id:06d}.png')
			camera_type = 'event_left_camera' if cam_name == 'event_cam00' else 'event_right_camera'
			camera = int_ext_loader.sensor_collection[f'{cam_prefix}{camera_type}']
			event_img, _ = rewrite_undistort_image(data_path_src, data_path_dst, camera) # return the distorted image

			# Event rendering
			ref_time = event_img_timestamps[cam_idx][mid[5 + cam_idx]]
			event_data = {
				'event_msg_timestamps': event_timestamps[cam_idx],
				'path': os.path.join(data_path, f'{cam_name}/event/data')
			}
			event_img_raw, event_img_render = render_events(event_img, event_data, camera, ref_time, dt_ev_img)
			cv2.imwrite(os.path.join(kitti360_path, f'{cam_name}/event/data', f'{frame_id:06d}.png'), event_img_raw)
			cv2.imwrite(os.path.join(kitti360_path, f'{cam_name}/event_render/data', f'{frame_id:06d}.png'), event_img_render)

print('Finish writing sensor data to kitti360-type dataset')

In [ ]:
# Description: Generate depth map
import cv2
import open3d as o3d
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

cmap = plt.get_cmap('jet')
cmap_colors = cmap(np.linspace(0, 1, 21)) * 255

frame_id = 0
##### Load point cloud and image
pcd_path = os.path.join(kitti360_path, 'ouster00_undistort/points/data', '{:06d}.pcd'.format(frame_id))
xyz_points = np.asarray(o3d.io.read_point_cloud(pcd_path).points)

if platform == 'vehicle':
  # images are already undistorted, but not rectified
  img_path = os.path.join(kitti360_path, 'vehicle_frame_cam00', 'image/data', '{:06d}.png'.format(frame_id)) 
  img = cv2.imread(img_path)
  camera = int_ext_loader.sensor_collection['vehicle_frame_left_camera']
else:
  # images are already undistorted, but not rectified
  img_path = os.path.join(kitti360_path, 'frame_cam00', 'image/data', '{:06d}.png'.format(frame_id)) 
  img = cv2.imread(img_path)
  camera = int_ext_loader.sensor_collection['frame_left_camera']

##### Load extrinsics from the tf_graph
T_cam_lidar = int_ext_loader.tf_graph.get_relative_transform(camera.frame_id, 'body_imu')
xyz_points_cam = np.matmul(T_cam_lidar[:3, :3], xyz_points.T).T + T_cam_lidar[:3, 3].T

##### Project point cloud onto the camera frame
for p_C in xyz_points_cam:
  flag, u_C = camera.project(p_C)
  if flag:
    i = int(min(p_C[2], 20.0))
    color = (int(cmap_colors[i, 0]), int(cmap_colors[i, 1]), int(cmap_colors[i, 2]))
    cv2.circle(img, (round(u_C[0]), round(u_C[1])), radius=1, color=color, thickness=-1)
img_pillow = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
display(img_pillow.resize((img_pillow.size[0] // 2, img_pillow.size[1] // 2))) 

In [ ]:
# TEST: visualize data
from PIL import Image
import open3d
from tools.utils import display_images_horizontally

# Visualize data
pcd_path = os.path.join(kitti360_path, 'ouster00_undistort/points/data', '{:06d}.pcd'.format(frame_id))
pcd = open3d.io.read_point_cloud(pcd_path)
print(pcd.points)

pcd_path = os.path.join(kitti360_path, 'ouster00/points/data', '{:06d}.pcd'.format(frame_id))
pcd = open3d.io.read_point_cloud(pcd_path)
print(pcd.points)

if platform == 'vehicle':
  img_path = os.path.join(kitti360_path, 'vehicle_frame_cam00', 'image/data', '{:06d}.png'.format(frame_id))
  img = Image.open(img_path)
  img_fl = img.resize((img.size[0] // 3, img.size[1] // 3))

  img_path = os.path.join(kitti360_path, 'vehicle_frame_cam01', 'image/data', '{:06d}.png'.format(frame_id))
  img = Image.open(img_path)
  img_fr = img.resize((img.size[0] // 3, img.size[1] // 3))
else:
  img_path = os.path.join(kitti360_path, 'frame_cam00', 'image/data', '{:06d}.png'.format(frame_id))
  img = Image.open(img_path)
  img_fl = img.resize((img.size[0] // 3, img.size[1] // 3))

  img_path = os.path.join(kitti360_path, 'frame_cam01', 'image/data', '{:06d}.png'.format(frame_id))
  img = Image.open(img_path)
  img_fr = img.resize((img.size[0] // 3, img.size[1] // 3))

img_path = os.path.join(kitti360_path, 'event_cam00', 'image/data', '{:06d}.png'.format(frame_id))
img = Image.open(img_path)
img_el = img.resize((img.size[0], img.size[1]))

img_path = os.path.join(kitti360_path, 'event_cam01', 'image/data', '{:06d}.png'.format(frame_id))
img = Image.open(img_path)
img_er = img.resize((img.size[0], img.size[1]))

display_images_horizontally(img_fl, img_fr, img_el, img_er)

In [ ]:
##### TEST: get extrinsics
print(int_ext_loader.tf_graph.get_relative_transform('body_imu', 'ouster00')[:3, :3])
print(int_ext_loader.tf_graph.get_relative_transform('body_imu', 'ouster00')[:3, 3])